In [3]:
# Downgrade pillow to avoid `UserWarning: Possibly corrupt EXIF data.`
!pip install pillow==4.0.0
!pip install keras

    100% |████████████████████████████████| 5.6MB 9.1MB/s eta 0:00:01
  Found existing installation: Pillow 5.2.0
    Uninstalling Pillow-5.2.0:
      Successfully uninstalled Pillow-5.2.0
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 378kB 33.3MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
%ls -la
%cd eye-Image-Analysis/
%ls -la

total 74568
drwxrwxr-x 14 ec2-user ec2-user     4096 Dec 16 20:19 ./
drwxr-xr-x  5 ec2-user ec2-user     4096 Dec 16 20:05 ../
drwxrwxr-x  5 ec2-user ec2-user     4096 Dec 13 01:48 Data/
drwxrwxr-x  4 ec2-user ec2-user     4096 Dec 13 01:48 Database/
drwxrwxr-x  5 ec2-user ec2-user     4096 Dec 13 01:48 Data_equalized/
drwxrwxr-x  3 ec2-user ec2-user     4096 Dec 13 01:48 Data_partial_excluded/
drwxrwxr-x  3 ec2-user ec2-user     4096 Dec 13 01:48 Data_Suspect/
drwxrwxr-x  7 ec2-user ec2-user     4096 Dec 13 13:04 efficientnet_keras_transfer_learning/
-rw-rw-r--  1 ec2-user ec2-user     1487 Dec 13 01:48 evaluation_test.py
drwxrwxr-x  8 ec2-user ec2-user     4096 Dec 16 20:20 .git/
-rw-rw-r--  1 ec2-user ec2-user     1212 Dec 13 01:48 .gitignore
-rw-rw-r--  1 ec2-user ec2-user     4859 Dec 13 01:48 Graphs.py
-rw-rw-r--  1 ec2-user ec2-user    15507 Dec 16 20:19 Interpreter.py
-rw-rw-r--  1 ec2-user ec2-user    11233 Dec 13 02:25 Interpreter.pyc
drwxrwxr-x  2 ec2-user ec2-user     4096 

In [14]:
from Interpreter import Interpreter

In [15]:
TARGET_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 100
IMAGE_SHAPE = (256, 256, 3)

inter = Interpreter(
    BATCH_SIZE,
    IMAGE_SHAPE,
    EPOCHS,
    TARGET_SIZE
)

train_images, validation_images, test_images = inter.split_data()

Found 706 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
Found 94 images belonging to 2 classes.


In [16]:
model, model_out = inter.train_efficient_net(
    train_images,
    test_images,
    validation_images
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Model)      (None, 8, 8, 1280)        4049564   
_________________________________________________________________
gap (GlobalMaxPooling2D)     (None, 1280)              0         
_________________________________________________________________
dropout_out (Dropout)        (None, 1280)              0         
_________________________________________________________________
fc_out (Dense)               (None, 2)                 2562      
Total params: 4,052,126
Trainable params: 4,010,110
Non-trainable params: 42,016
_________________________________________________________________
This is the number of trainable layers before freezing the conv base: 213
This is the number of trainable layers after freezing the conv base: 2


NameError: name 'batch_size' is not defined

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import glob
import shutil
from IPython.display import Image
import os
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
%ls -la
%cd efficientnet_keras_transfer_learning/

total 74560
drwxrwxr-x 14 ec2-user ec2-user     4096 Dec 16 19:32 ./
drwxr-xr-x  5 ec2-user ec2-user     4096 Dec 16 18:26 ../
drwxrwxr-x  5 ec2-user ec2-user     4096 Dec 13 01:48 Data/
drwxrwxr-x  4 ec2-user ec2-user     4096 Dec 13 01:48 Database/
drwxrwxr-x  5 ec2-user ec2-user     4096 Dec 13 01:48 Data_equalized/
drwxrwxr-x  3 ec2-user ec2-user     4096 Dec 13 01:48 Data_partial_excluded/
drwxrwxr-x  3 ec2-user ec2-user     4096 Dec 13 01:48 Data_Suspect/
drwxrwxr-x  7 ec2-user ec2-user     4096 Dec 13 13:04 efficientnet_keras_transfer_learning/
-rw-rw-r--  1 ec2-user ec2-user     1487 Dec 13 01:48 evaluation_test.py
drwxrwxr-x  8 ec2-user ec2-user     4096 Dec 16 19:33 .git/
-rw-rw-r--  1 ec2-user ec2-user     1212 Dec 13 01:48 .gitignore
-rw-rw-r--  1 ec2-user ec2-user     4859 Dec 13 01:48 Graphs.py
-rw-rw-r--  1 ec2-user ec2-user    15507 Dec 16 19:20 Interpreter.py
-rw-rw-r--  1 ec2-user ec2-user    11233 Dec 13 02:25 Interpreter.pyc
drwxrwxr-x  2 ec2-user ec2-user     4096 

In [16]:
# Options: EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3
# Higher the number, the more complex the model is.
from efficientnet import EfficientNetB0 as Net
from efficientnet import center_crop_and_resize, preprocess_input

In [20]:
TARGET_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 100
IMAGE_SHAPE = (256, 256, 3)

In [21]:
# loading pretrained conv base model
model = Net(
    weights='imagenet',
    include_top=False,
    input_shape=IMAGE_SHAPE
)


In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_65 (Conv2D)              (None, 128, 128, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1_49 (Batc (None, 128, 128, 32) 128         conv2d_65[0][0]                  
__________________________________________________________________________________________________
swish_49 (Swish)                (None, 128, 128, 32) 0           batch_normalization_v1_49[0][0]  
__________________________________________________________________________________________________
depthwise_

In [23]:
sys.path.append('..')

In [24]:
%ls -la

total 216
drwxrwxr-x 6 ec2-user ec2-user   4096 Dec 13 13:10 ./
drwxrwxr-x 7 ec2-user ec2-user   4096 Dec 13 13:10 ../
drwxrwxr-x 3 ec2-user ec2-user   4096 Dec 13 13:14 efficientnet/
drwxrwxr-x 2 ec2-user ec2-user   4096 Dec 13 13:10 examples/
drwxrwxr-x 8 ec2-user ec2-user   4096 Dec 13 13:10 .git/
-rw-rw-r-- 1 ec2-user ec2-user   1274 Dec 13 13:10 .gitignore
-rw-rw-r-- 1 ec2-user ec2-user     27 Dec 13 13:10 __init__.py
-rw-rw-r-- 1 ec2-user ec2-user 173397 Dec 13 13:10 Keras_efficientnet_transfer_learning.ipynb
-rw-rw-r-- 1 ec2-user ec2-user     17 Dec 13 13:10 MANIFEST.in
drwxrwxr-x 2 ec2-user ec2-user   4096 Dec 13 13:10 misc/
-rw-rw-r-- 1 ec2-user ec2-user   3626 Dec 13 13:10 README.md
-rw-rw-r-- 1 ec2-user ec2-user   3700 Dec 13 13:10 setup.py


In [28]:
%cd ../../..

/home/ec2-user/SageMaker/eye-Image-Analysis


In [30]:
%ls -la

total 74496
drwxrwxr-x 14 ec2-user ec2-user     4096 Dec 16 17:43 ./
drwxr-xr-x  5 ec2-user ec2-user     4096 Dec 16 17:26 ../
drwxrwxr-x  5 ec2-user ec2-user     4096 Dec 13 01:48 Data/
drwxrwxr-x  4 ec2-user ec2-user     4096 Dec 13 01:48 Database/
drwxrwxr-x  5 ec2-user ec2-user     4096 Dec 13 01:48 Data_equalized/
drwxrwxr-x  3 ec2-user ec2-user     4096 Dec 13 01:48 Data_partial_excluded/
drwxrwxr-x  3 ec2-user ec2-user     4096 Dec 13 01:48 Data_Suspect/
drwxrwxr-x  7 ec2-user ec2-user     4096 Dec 13 13:04 efficientnet_keras_transfer_learning/
-rw-rw-r--  1 ec2-user ec2-user     1487 Dec 13 01:48 evaluation_test.py
drwxrwxr-x  8 ec2-user ec2-user     4096 Dec 16 17:43 .git/
-rw-rw-r--  1 ec2-user ec2-user     1212 Dec 13 01:48 .gitignore
-rw-rw-r--  1 ec2-user ec2-user     4859 Dec 13 01:48 Graphs.py
-rw-rw-r--  1 ec2-user ec2-user    13648 Dec 16 17:33 Interpreter.py
-rw-rw-r--  1 ec2-user ec2-user    11233 Dec 13 02:25 Interpreter.pyc
drwxrwxr-x  2 ec2-user ec2-user     4096 

In [ ]:
from Interpreter import Interpreter

In [44]:
inter = Interpreter(
    BATCH_SIZE,
    IMAGE_SHAPE,
    EPOCHS,
    TARGET_SIZE
)

In [45]:
train_images, validation_images, test_images = inter.split_data()

Found 706 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
Found 94 images belonging to 2 classes.


In [37]:
%cd efficientnet_keras_transfer_learning/

/home/ec2-user/SageMaker/eye-Image-Analysis/efficientnet_keras_transfer_learning


In [38]:
# loading pretrained conv base model
conv_base = Net(
    weights='imagenet',
    include_top=False,
    input_shape=IMAGE_SHAPE
)

In [39]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalMaxPooling2D(name="gap"))
# model.add(layers.Flatten(name="flatten"))
model.add(layers.Dropout(0.2, name="dropout_out"))
# model.add(layers.Dense(256, activation='relu', name="fc1"))
model.add(layers.Dense(2, activation='softmax', name="fc_out"))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [40]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Model)      (None, 8, 8, 1280)        4049564   
_________________________________________________________________
gap (GlobalMaxPooling2D)     (None, 1280)              0         
_________________________________________________________________
dropout_out (Dropout)        (None, 1280)              0         
_________________________________________________________________
fc_out (Dense)               (None, 2)                 2562      
Total params: 4,052,126
Trainable params: 4,010,110
Non-trainable params: 42,016
_________________________________________________________________


In [41]:
print('This is the number of trainable layers '
      'before freezing the conv base:', len(model.trainable_weights))

conv_base.trainable = False

print('This is the number of trainable layers '
      'after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable layers before freezing the conv base: 213
This is the number of trainable layers after freezing the conv base: 2


In [42]:
% cd ..
% ls

/home/ec2-user/SageMaker/eye-Image-Analysis
Data/                                  model_simple_64.h5
Database/                              model_simple_64.json
Data_equalized/                        model_simple.h5
Data_partial_excluded/                 model_simple.json
Data_Suspect/                          pre_proc.py
efficientnet_keras_transfer_learning/  Processing/
evaluation_test.py                     Processor.py
Graphs.py                              __pycache__/
Interpreter.py                         README.md
Interpreter.pyc                        references/
model1_accuracy_128.png                requirements.txt
model1_accuracy_64.png                 test_interpreter.py
model1_accuracy.png                    test_processor.py
model1_loss_128.png                    train_model.ipynb
model1_loss_64.png                     train_start.py
model1_loss.png                        train_transf_net.ipynb
model.h5                               Untitled.ipynb
model_simple_128.h5  

In [43]:
model.compile(
#     loss='categorical_crossentropy',
    loss='sparse_categorical_crossentropy',
    optimizer=optimizers.RMSprop(lr=2e-5),
    metrics=['acc']
)
history = model.fit_generator(
      train_images,
      steps_per_epoch= len(train_images.classes) //batch_size,
      epochs=epochs,
      validation_data=validation_images,
      validation_steps= len(validation_images.classes) //batch_size,
#       verbose=1,
#       use_multiprocessing=True,
      workers=2
)

Epoch 1/100


KeyboardInterrupt: 